In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import numpy as np
import pandas as pd

In [22]:
import xarray as xr

In [23]:
from file_processing import get_netcdf_files, get_rhime_outs

In [24]:
from country_totals import get_country_trace, get_x_to_country_mat

In [11]:
#!pip install sparse

In [15]:
files = get_netcdf_files("/home/brendan/Documents/inversions/plotting/sf6_best")

In [16]:
outs = get_rhime_outs(files)

In [18]:
countries = xr.open_dataset("/home/brendan/Documents/inversions/openghg_inversions/countries/country_EUROPE.nc")

In [19]:
countries_ukmo = xr.open_dataset("/home/brendan/Documents/inversions/openghg_inversions/countries/country-ukmo_EUROPE.nc")

In [25]:
country_traces = [get_country_trace(countries, "sf6", hbmcmc_outs=ds) for ds in outs]

In [26]:
country_traces_ukmo = [get_country_trace(countries_ukmo, "sf6", hbmcmc_outs=ds) for ds in outs]

In [27]:
country_traces[0]

<xarray.DataArray (ncountries: 104, steps: 10000)>
array([[4.71633334e+07, 3.59968995e+07, 2.85150035e+08, ...,
        4.94568652e+07, 1.04207772e+08, 1.13004711e+08],
       [4.67104970e+05, 7.38125094e+05, 2.04443474e+06, ...,
        3.63534044e+06, 6.39277158e+05, 1.19529518e+06],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.02697079e+02, 1.86238591e+02, 4.33239373e+03, ...,
        2.27562294e+02, 1.41590098e+03, 1.49408703e+03],
       [1.34835277e+07, 8.29592807e+06, 1.92984851e+08, ...,
        1.01366769e+07, 6.30707772e+07, 6.65535454e+07],
       [3.46808157e+02, 2.13378546e+02, 4.96373963e+03, ...,
        2.60724220e+02, 1.62223570e+03, 1.71181556e+03]])
Coordinates:
    country  (ncountries) <U52 ...
    steps    (steps) int64 ...
Dimensions without coordinates: ncountries

In [28]:
outs[0]

<xarray.Dataset>
Dimensions:            (nmeasure: 877, nUI: 2, steps: 10000, nparam: 97,
                        nBC: 16, nsigma_site: 8, nsigma_time: 4, nsite: 6,
                        lat: 293, lon: 391, countrynames: 20, nlatent: 97)
Coordinates:
    steps              (steps) int64 ...
    nlatent            (nlatent) int64 ...
    nBC                (nBC) int64 ...
    nmeasure           (nmeasure) int64 ...
    nUI                (nUI) int64 ...
    nsites             (nsite) int64 ...
  * nsigma_time        (nsigma_time) int64 0 1 2 3
  * nsigma_site        (nsigma_site) int64 0 1 2 3 4 5 6 7
  * lat                (lat) float32 10.73 10.96 11.2 ... 78.59 78.82 79.06
  * lon                (lon) float32 -97.9 -97.55 -97.2 ... 38.68 39.03 39.38
  * countrynames       (countrynames) <U14 'None' 'RestEU' ... 'UNITED KINGDOM'
  * nparam             (nparam) int64 0 1 2 3 4 5 6 7 ... 90 91 92 93 94 95 96
Dimensions without coordinates: nsite
Data variables: (12/43)
    Yobs               (nmeasure) float64 ...
    Yerror             (nmeasure) float64 ...
    Ytime              (nmeasure) datetime64[ns] ...
    Yapriori           (nmeasure) float64 ...
    Ymodmean           (nmeasure) float64 ...
    Ymodmedian         (nmeasure) float64 ...
    ...                 ...
    country68          (countrynames, nUI) float64 ...
    country95          (countrynames, nUI) float64 ...
    countryapriori     (countrynames) float64 ...
    countrydefinition  (lat, lon) int32 ...
    xsensitivity       (nmeasure, nparam) float64 ...
    bcsensitivity      (nmeasure, nBC) float64 ...
Attributes: (12/15)
    Start date:           2020-06-01
    End date:             2020-07-01
    Latent sampler:       mc.nuts.NUTS 
    Hyper sampler:        licer.Slice o
    Burn in:              10000
    Tuning steps:         10000
    ...                   ...
    Model error Prior:    pdf,uniform,lower,0.1,upper,1.0
    BCs Prior:            pdf,truncatednormal,mu,1.0,sigma,0.03,lower,0.0
    Creator:              bm13805
    Date created:         2024-01-09 10:33:53.036470
    Convergence:          Passed
    Repository version:

In [30]:
country_traces = [trace.expand_dims({"time": [out_ds.Ytime.min().values]}) for trace, out_ds in zip(country_traces, outs)]
country_trace_ds = xr.concat(country_traces, dim="time")

In [32]:
country_traces_ukmo = [trace.expand_dims({"time": [out_ds.Ytime.min().values]}) for trace, out_ds in zip(country_traces_ukmo, outs)]
country_trace__ukmo_ds = xr.concat(country_traces_ukmo, dim="time")

In [31]:
country_trace_ds

<xarray.DataArray (time: 11, ncountries: 104, steps: 10000)>
array([[[4.71633334e+07, 3.59968995e+07, 2.85150035e+08, ...,
         4.94568652e+07, 1.04207772e+08, 1.13004711e+08],
        [4.67104970e+05, 7.38125094e+05, 2.04443474e+06, ...,
         3.63534044e+06, 6.39277158e+05, 1.19529518e+06],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [3.02697079e+02, 1.86238591e+02, 4.33239373e+03, ...,
         2.27562294e+02, 1.41590098e+03, 1.49408703e+03],
        [1.34835277e+07, 8.29592807e+06, 1.92984851e+08, ...,
         1.01366769e+07, 6.30707772e+07, 6.65535454e+07],
        [3.46808157e+02, 2.13378546e+02, 4.96373963e+03, ...,
         2.60724220e+02, 1.62223570e+03, 1.71181556e+03]],

       [[6.02135885e+07, 4.92280239e+07, 1.03657492e+08, ...,
         1.03459793e+08, 3.24321801e+07, 7.37626725e+07],
        [1.82895743e+05, 1.75496631e+05, 2.76449719e+05, ...,
         1.97775303e+06, 6.49091181e+05, 2.18893836e+05],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
...
        [3.06949187e+03, 3.90185458e+03, 2.68088136e+03, ...,
         3.58102685e+03, 5.13508236e+03, 4.38439355e+03],
        [1.44302572e+08, 1.83433505e+08, 1.26033263e+08, ...,
         1.68350791e+08, 2.41409857e+08, 2.06118567e+08],
        [3.52081809e+03, 4.47556820e+03, 3.07506779e+03, ...,
         4.10756719e+03, 5.89012502e+03, 5.02905782e+03]],

       [[5.49629162e+07, 3.40656345e+07, 2.17508165e+08, ...,
         7.51306189e+07, 4.67109448e+07, 9.73504582e+07],
        [4.57656641e+05, 4.24261946e+05, 5.34712787e+06, ...,
         9.64924835e+05, 2.73917774e+05, 1.31093528e+06],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [4.72336566e+02, 8.28164430e+01, 2.65116675e+03, ...,
         9.60909775e+02, 4.01610759e+02, 1.21138248e+03],
        [2.22054282e+07, 3.89335637e+06, 1.24636323e+08, ...,
         4.51741715e+07, 1.88804753e+07, 5.69493635e+07],
        [5.41787109e+02, 9.49934526e+01, 3.04098406e+03, ...,
         1.10219823e+03, 4.60662052e+02, 1.38949948e+03]]])
Coordinates:
  * time     (time) datetime64[ns] 2020-06-01 2020-07-01 ... 2021-04-01
    country  (ncountries) <U52 'OCEAN' 'VENEZUELA' ... 'ALGERIA' 'ALBANIA'
  * steps    (steps) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
Dimensions without coordinates: ncountries

In [ ]:
country_results = xr.merge(